In [1]:
import os
import glob
import json
from transformers import PreTrainedTokenizerFast




/home/arturo_hernandez/anaconda3/envs/phylm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:

tokenizer = PreTrainedTokenizerFast(tokenizer_file="../data/tokenizer/tokenizer.json")

In [3]:
raw_data_path= '../data/raw/'
processed_data_path='../data/processed/'
file_list= glob.glob(os.path.join(raw_data_path,'*.txt'))
for file in file_list:
    title=file.split('/')[-1].split('.')[0]
    text_file = open(file, "r")
    indi_list=tokenizer.encode(text_file.read())
    text_file.close()
    with open(os.path.join(processed_data_path,title+'.json'), "w") as save_file:
        json.dump({'indices':indi_list}, save_file)
